In [1]:
import numpy as np
import torch
from torch import Tensor


In [2]:
import os
import sys
# sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
# print(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))

In [3]:
# sys.path.append(os.path.join(os.getcwd(), '..'))
os.chdir(os.path.join(os.getcwd(), '..'))

In [4]:
os.listdir(os.getcwd())

['.git',
 '.gitignore',
 '.ruff_cache',
 '.venv',
 'baselines',
 'BM_2Sphere',
 'build',
 'CharTrajectories',
 'CIFAR10',
 'data',
 'development',
 'example.py',
 'harmonic',
 'images',
 'img',
 'Nbody',
 'Path_Development_Net.egg-info',
 'pdev.egg-info',
 'README.md',
 'setup.py',
 'sMNIST',
 'SpeechCommands',
 'test',
 '__init__.py']

In [5]:
import torch 
# from development.gl import gl
from development.nn import development_layer
from development.gl import gl
from development.he import he


In [6]:
def generate_BM_paths_torch(
    N: int, delta: float, rho: float, t: float = 1, device: str = "cuda"
) -> Tensor:
    """
    Return N samples of 2D brownian motion with step-size delta and sample correlation rho
    """
    k = int(np.floor(t / delta))
    sigma = np.sqrt(delta)
    x = sigma * torch.randn((1, k, 2)).repeat(N, 1, 1)
    y = sigma * torch.randn((N, k, 2))
    z = rho * x + np.sqrt(1 - rho**2) * y
    z = torch.cumsum(z, axis=1)
    z = torch.cat([torch.zeros((N, 1, 2)), z], axis=1)
    return z

In [7]:
paths = generate_BM_paths_torch(100, 0.01, .2)
print(paths.shape)



torch.Size([100, 101, 2])


In [8]:
hidden_size = 4
dev_layer = development_layer(
    input_size=2,
    hidden_size=hidden_size,
    channels=1,
    param=he,
    complexification=False,
    return_sequence=False,
)

In [9]:
dev_layer

development_layer(
  (projection): projection(
    (param_map): he()
  )
)

In [10]:
devs = dev_layer(paths)
print(devs.shape)


torch.Size([100, 1, 4, 4])


In [11]:
devs[0]

tensor([[[ 1.0000,  0.3210,  0.0615,  0.0683],
         [ 0.0000,  1.0000, -0.3581,  0.0433],
         [ 0.0000,  0.0000,  1.0000,  0.2368],
         [ 0.0000,  0.0000,  0.0000,  1.0000]]], grad_fn=<SelectBackward0>)